In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score, precision_score, f1_score

In [2]:
# Load data
cluster_data = pd.read_csv("./dataset_cluster_1.csv")

In [5]:
# 좌표 데이터와 Venue ID 추출
coord = cluster_data[['Latitude', 'Longitude']]
venue_ids = cluster_data['Venue ID'].tolist()

In [6]:
# KDE 학습
kde = KernelDensity(bandwidth=0.01, metric='haversine', kernel='gaussian')
kde.fit(coord)

KernelDensity(bandwidth=0.01, metric='haversine')

In [7]:

# 추천 POI 리스트 생성
k = 10
target_user_coord = [[40.7589,-73.9851]] # 샘플 좌표, 실제 데이터에는 없는 좌표
poi_scores = kde.score_samples(coord)
recommended_poi_indices = poi_scores.argsort()[::-1][:k]
recommended_poi_ids = [venue_ids[i] for i in recommended_poi_indices]

In [8]:
# 정밀도, 재현율, F1 평균 계산
avg_precision = 0
avg_recall = 0
avg_f1 = 0
num_clusters = cluster_data['Cluster'].nunique()

In [9]:
for cluster_id in range(1, num_clusters + 1):
    # 라벨링1인 클러스터 추출
    cluster = cluster_data[cluster_data['Cluster']==cluster_id]
    
    # 좌표 데이터와 Venue ID 추출
    coord = cluster[['Latitude', 'Longitude']]
    venue_ids = cluster['Venue ID'].tolist()
    
    # KDE 학습
    kde = KernelDensity(bandwidth=0.01, metric='haversine', kernel='gaussian')
    kde.fit(coord)
    
    # 추천 POI 리스트 생성
    target_user_coord = [[cluster['Latitude'].iloc[0], cluster['Longitude'].iloc[0]]]
    poi_scores = kde.score_samples(coord)
    recommended_poi_indices = poi_scores.argsort()[::-1][:k]
    recommended_poi_ids = [venue_ids[i] for i in recommended_poi_indices]
    
    # 정밀도, 재현율, F1 평균 계산
    true_poi_ids = cluster['Venue ID'].tolist()
    precision = precision_score(true_poi_ids, recommended_poi_ids, zero_division=0)
    recall = recall_score(true_poi_ids, recommended_poi_ids, zero_division=0)
    f1 = f1_score(true_poi_ids, recommended_poi_ids, zero_division=0)
    
    avg_precision += precision
    avg_recall += recall
    avg_f1 += f1


ValueError: Found input variables with inconsistent numbers of samples: [3009, 10]